## Importing the libraries

In [2]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.factorization import KMeans

## Ignore all GPUs
tf k-means does not benefit from it.

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

## Loading the MNIST images data

In [5]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
full_data_x = mnist.train.images
print('Data has %d datapoints' % len(full_data_x))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Data has 55000 datapoints


## Defining the hyperparameters

In [6]:
# Parameters
num_steps = 100  # Total steps to train
k = 50  # The number of clusters
num_classes = 10  # The 10 digits
num_features = full_data_x.shape[1]  # Each image is 28x28 pixels

## Defining the placeholders for input data

In [7]:
# Input images
X = tf.placeholder(tf.float32, shape=[None, num_features])
# Labels (for assigning a label to a centroid and testing)
Y = tf.placeholder(tf.float32, shape=[None, num_classes])

## Initializing the KMeans algorithm and its scope

In [9]:
# K-Means Parameters
kmeans = KMeans(inputs=X, num_clusters=k, distance_metric='cosine')
# Build KMeans graph
with tf.name_scope('KMeans') as scope:
    training_graph = kmeans.training_graph()

## Defining the average distance operation and its summary 

In [10]:
(all_scores, cluster_idx, scores, cluster_centers_initialized, init_op, train_op) = training_graph
cluster_idx = cluster_idx[0]  # fix for cluster_idx being a tuple
avg_distance = tf.reduce_mean(scores)
distance_h = tf.summary.scalar('average_distance', avg_distance) 

# Initialize the variables (i.e. assign their default value)
init_vars = tf.global_variables_initializer()
merged_summary_op = tf.summary.merge_all()

## Starting a TF session and initalizing summary data

In [11]:
# Start TensorFlow session
session = tf.Session()

# Run the initializer
session.run(init_vars, feed_dict={X: full_data_x})
session.run(init_op, feed_dict={X: full_data_x})
summary_writer = tf.summary.FileWriter('logs', graph=session.graph)

## Training the KMeans on input data and collecting the summary

In [12]:
# Training
for i in range(num_steps):
    _, d, idx = session.run([train_op, avg_distance, cluster_idx], feed_dict={X: full_data_x})
    summary_info = session.run(merged_summary_op, feed_dict={X: full_data_x})
    summary_writer.add_summary(summary_info, i)
    if i % 10 == 0:
        print("Step %i, Avg Distance: %f" % (i+1, d))

Step 1, Avg Distance: 0.314557
Step 11, Avg Distance: 0.192617
Step 21, Avg Distance: 0.191365
Step 31, Avg Distance: 0.191102
Step 41, Avg Distance: 0.191048
Step 51, Avg Distance: 0.190969
Step 61, Avg Distance: 0.190873
Step 71, Avg Distance: 0.190795
Step 81, Avg Distance: 0.190749
Step 91, Avg Distance: 0.190730


## Assigning labels to clusters and definding accuracy operation

In [13]:
# Assign a label to each centroid
# Count total number of labels per centroid, using the label of each training
# sample to their closest centroid (given by 'idx')
counts = np.zeros(shape=(k, num_classes))
for i in range(len(idx)):
    counts[idx[i]] += mnist.train.labels[i]
# Assign the most frequent label to the centroid
labels_map = [np.argmax(c) for c in counts]
labels_map = tf.convert_to_tensor(labels_map)

In [14]:
# Evaluation ops
# Lookup: centroid_id -> label
cluster_label = tf.nn.embedding_lookup(labels_map, cluster_idx)
# Compute accuracy
correct_prediction = tf.equal(cluster_label, tf.cast(tf.argmax(Y, 1), tf.int32))
accuracy_op = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## Using the accuracy operation to check accuracy of model on test data

In [15]:
# Test Model
test_x, test_y = mnist.test.images, mnist.test.labels
print("Test Accuracy:", session.run(accuracy_op, feed_dict={X: test_x, Y: test_y}))

Test Accuracy: 0.8297


## Tensor flows
![graph.png](graph.png)